## Árbol de decisión

Un modelo el cual utiliza varios niveles, donde en cada nivel existe una caractrística que se evalua a través de un umbral, va dividiendo los datos en los que son mayor o menor al umbral definido en la característica.


### Regresión

Este algoritmo usa todas las variables y los splits posibles (umbrales) y elige el umbral en cada variable que reduzca en mayor medida la varianza.


$$
\text{Reducción de varianza} = \text{Varianza total} - \left( \frac{N_1}{N} \times \text{Var}(G1) + \frac{N_2}{N} \times \text{Var}(G2) \right)
$$

### Clasificación

De igual forma que en la regresión, el modelo usa todas las variables y prueba todos los umbrales posibles, eligiendo el que cause una mayor reducción de impureza, la cual se calcula a través del Gini o de la Entriopía.

$$
Gini = 1 - \sum p_i^2
$$

$$
Entropía = -\sum p_i \log_2 p_i
$$


$$
\text{Ganancia de Impureza} = \text{Impureza Nodo Padre} - \left( \frac{N_1}{N} \times \text{Impureza}(G1) + \frac{N_2}{N} \times \text{Impureza}(G2) \right)
$$

---

## Random Forest

Utilizas la técnica de bootsrap para hacer muestreos aleatorios de tus datos, sobre los cuales entrenas tu modelo de árbol de decisión. Cada uno de estos modelos realiza sus predicciones, y el promedio de estas sería la predicción utilizada por el algoritmo de Random Forest.

---




